In [56]:
# for cpu
# %pip install "qdrant-client[fastembed]"

# for gpu
# %pip install 'qdrant-client[fastembed-gpu]'

In [57]:
from pathlib import Path
import json

product_path = "../training-sets/MEN_FASHION_WITH_REVIEWS"
product_list = json.loads(Path(product_path + "/raw_products.json").read_text())
products = product_list[0:50]


In [58]:
import torch # type: ignore

# from imagebind import data
# from imagebind.models import imagebind_model
# from imagebind.models.imagebind_model import ModalityType


# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "cpu"

# # Instantiate model
# model = imagebind_model.imagebind_huge(pretrained=True)
# model.eval()
# model.to(device)

In [59]:
from urllib.parse import urlparse
def _get_product_text_data(product: dict):
    
    description = product.get("description", None)
    revised_description = product.get("description", None)
    name = product.get("name", None)
    product_page_url = product.get("product_page_url", None)
    
    
    if description is None or name is None or product_page_url is None:
        raise ValueError("description, name or product_page_url is missing")
    
    
    parsed_url = urlparse(product_page_url)
    product_url_path = parsed_url.path
    
    text_to_embded = f"{name}\n{description}\n{product_url_path}"
    
    return text_to_embded

In [60]:


from sentence_transformers import SentenceTransformer
def embed_product_text_data(product: dict):
    product_text_data = _get_product_text_data(product)
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    embeddings = model.encode(product_text_data)
    return embeddings    
    
# embed_product_text_data(products[0])

In [61]:
images_directory = "../training-sets/MEN_FASHION_WITH_REVIEWS/images/"

In [62]:
from transformers import ViTModel, ViTImageProcessor
from PIL import Image

def embed_product_image(product: dict):
   
    product_asin = product.get("product_asin", None)
        
    if product_asin is None:
        raise ValueError("product_asin is missing")
    
    path_to_image = Path(f"{images_directory}/{product_asin}.png")
    if not path_to_image.exists():
        raise FileNotFoundError(f"{path_to_image} does not exist.")
    

    image = Image.open(path_to_image.absolute())

    model = ViTModel.from_pretrained("google/vit-base-patch16-224")
    image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224") # type: ignore

    # Load and process image
    inputs = image_processor(images=image, return_tensors="pt")

    # Generate embedding
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0].detach().numpy()
    
    return embedding[0]
    
# embed_product_image(products[0])

### Embed the product text data

In [63]:
from qdrant_client import QdrantClient, models

client = QdrantClient(location=":memory:")

encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
client.recreate_collection(
    collection_name="product_text_data",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # type: ignore | Vector size is defined by used model
        distance=models.Distance.COSINE),
)

client.upload_records(
    collection_name="product_text_data",
    records=[
        models.Record(
            id=idx, 
            vector=encoder.encode(_get_product_text_data(product)),  # type: ignore
            payload=product
        ) for idx, product in enumerate(products)
    ]
)


/tmp/ipykernel_1946360/1684884611.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
/tmp/ipykernel_1946360/1684884611.py:13: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  client.upload_records(


### Perform a search on the text

In [64]:
# Let's now search for something

hits = client.search(
    collection_name="product_text_data",
    query_vector=encoder.encode("Do you have any men sneakers I can wear for the winter?"),
    limit=1
)
for hit in hits:
  print("score:", hit.score, hit.payload )

score: 0.45414936542510986 {'id': 'fde9e6c3-6ae2-40c6-b9bf-27c726ebfe9b', 'name': 'Wide Fit Trainers for Men Arch Support Road Running Athletic Sneakers Breathable Gym Shoes for Fitness Jogging Walking - Fresh Core', 'description': "    Product details     Care instructions     Machine Wash       Sole material     Rubber       Outer material     3-Layer Mesh       Inner material     Mesh      About this item   Breathable Knit Upper: The upper of mens trainer is made of mesh and synthetic materials. The knitted fabric is comfortable, breathable and lightweight to keep your feet dry during exercise   Wide Toe Box: These wide trainers are good for wide feet and people suffering from foot problems like plantar fasciitis, foot pain, flat feet, bunions and hammertoes   Slip On Design: Men's slip on trainers with elastic and soft synthetic lining protect your back heel from abrasion, which is convenient to put on and take off   Cushioned Midsole: The EVA midsole provides effective shock absor

### Embed the product images

In [65]:
from qdrant_client import QdrantClient, models

client = QdrantClient(location=":memory:")

encoder = ViTModel.from_pretrained("google/vit-base-patch16-224")


client.recreate_collection(
    collection_name="product_image_data",
    vectors_config=models.VectorParams(
        size=768, # Vector size is defined by used model
        distance=models.Distance.COSINE),
)

client.upload_records(
    collection_name="product_image_data",
    records=[
        models.Record(
            id=idx, 
            vector=embed_product_image(product),  
            payload=product
        ) for idx, product in enumerate(products)
    ]
)

print(embed_product_image(products[0]))


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1946360/3933785330.py:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', '

### Perform a search on the product image

In [66]:

print(embed_product_image(products[0]))

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[ 6.94684505e-01  2.15739340e-01  5.45185745e-01 -3.38562161e-01
 -1.41485143e+00  4.42166388e-01  7.17486218e-02  3.15685302e-01
  3.84141535e-01  1.25278091e+00 -8.70135188e-01 -6.61743343e-01
 -2.00026536e+00 -6.79830849e-01 -9.23880339e-01  5.64478457e-01
 -7.94525027e-01 -3.11493665e-01 -3.75945009e-02  2.32250556e-01
  6.76313043e-02 -4.54676211e-01  6.02878511e-01 -3.98378283e-01
  1.20202398e+00 -8.75125468e-01  2.99798042e-01  8.92604172e-01
  3.51383299e-01 -9.51291859e-01  1.93554252e-01 -1.82681456e-01
 -7.15947747e-01 -1.86004543e+00  1.30573884e-01  1.91586167e-01
 -7.49744356e-01 -9.73309278e-01 -1.71413600e+00 -1.79768121e+00
  2.01225340e-01  5.04286662e-02  4.35617656e-01  8.23915958e-01
  8.28876078e-01  1.30129015e+00 -1.63749778e+00 -1.72240102e+00
  1.93475437e+00  6.44203305e-01 -1.86320648e-01  1.02748275e+00
 -4.67510134e-01  1.01547873e+00  2.12661815e+00 -3.11534166e-01
  6.91120863e-01 -1.36724129e-01  3.78378659e-01  5.00444829e-01
 -6.65137589e-01 -7.07516

In [71]:

# Let's now search for something
expected_product = products[0]

print("Expected Product", expected_product['name'], "\n")

hits = client.search(
    collection_name="product_image_data",
    query_vector=embed_product_image(product=expected_product),
    limit=3
)
for hit in hits:
  print("\ncore:", hit.score, hit.payload )

Expected Product Safety Trainers Men Women Steel Toe Cap Trainers Safety Shoes Lightweight Work Shoes Safety Boots Industrial Protective Shoes 



Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



core: 0.9999998807907104 {'id': 'b9b7b87e-0786-4842-9d32-252aec8eb314', 'name': 'Safety Trainers Men Women Steel Toe Cap Trainers Safety Shoes Lightweight Work Shoes Safety Boots Industrial Protective Shoes', 'description': "    Product details     Care instructions     Hand Wash Only       Sole material     Thermoplastic Elastomers       Outer material     Textile       Inner material     Textile      About this item   Breathable upper: Fashion appearance, mesh hole design very breathable, these work boots are extremely light, while the breathable mesh upper can effectively disperse heat, even in hot working conditions, your feet will stay fresh cool and odorless.   Anti-Smashing: The standard widened steel toe is able absorbe forces of 200J and compression of 15000N, Not only can it better protect your feet from the falling and rolling heavy objects, but also provide plenty of inner space for your toes, making your feet more comfortable.   Puncture Proof: The safety trainers with th